In [61]:
%run ../Python_files/load_dicts.py
%run ../Python_files/util.py

In [62]:
from util import *
import numpy as np
from numpy.linalg import inv, matrix_rank
import json

In [63]:
# # load logit_route_choice_probability_matrix
# P = zload('../temp_files/logit_route_choice_probability_matrix_Sioux.pkz')
# P = np.matrix(P)

# print('rank of P is: ')
# print(matrix_rank(P))

# print('shape of P is: ')
# print(np.shape(P))

In [64]:
# # load path-link incidence matrix
# A = zload('../temp_files/path-link_incidence_matrix_Sioux-Falls.pkz')

# print('rank of A is: ')
# print(matrix_rank(A))

# print('shape of A is: ')
# print(np.shape(A))

In [65]:
# load link counts data

flow_list = []
with open('SiouxFallsFlow.txt', 'r') as f:
    read_data = f.readlines()
    flag = 0
    for row in read_data:
        flag += 1
        if flag > 1:
            flow_list.append(float(row.split('\t')[2]))

x_0 = np.array(flow_list)

In [66]:
x_0

array([  4494.65764646,   8119.07994805,   4519.07994805,   5967.33639617,
         8094.65764646,  14006.37101986,  10022.31961516,  14030.5609174 ,
        18006.37101986,   5200.        ,  18030.5609174 ,   8798.26771411,
        15780.78205547,   5991.75869776,   8806.49866681,  12492.92536056,
        12101.52912231,  15794.01060698,  12525.57861486,  12040.91827285,
         6882.66491266,   8388.713063  ,  15796.7410003 ,   6836.70597529,
        21744.07608018,  21814.07608764,  17726.62503296,  23125.7972901 ,
        11047.09388127,   8100.        ,   5300.        ,  17604.22353323,
         8365.28565386,   9776.11953275,   9973.70741603,   8404.93462395,
        12287.60526902,  12378.64203998,  11121.35796002,   9814.06906293,
         9036.33413403,   8400.43683027,  23192.28335936,   9079.82031659,
        19083.28976475,  18409.93502652,   8406.71440521,  11073.00931921,
        11695.00291653,  15278.32524152,   8100.        ,  11683.83828244,
         9953.02143205,  

### Assignment Equation

We have the following equation: 
$$AP'\boldsymbol{\lambda} = \textbf{x},$$
whose least-squares solution can be written as
$$\boldsymbol{\lambda} = (AP')^+\textbf{x}, \quad (1)$$
where $(AP')^{+}$ is the pseudo-inverse of $AP'$.

However, the $\boldsymbol{\lambda}$ given by (1) might contain negative entries, which is not desired. Thus, instead, we solve a constrained least-squares problem:
$$\mathop {\min }\limits_{\boldsymbol{\lambda}  \geq \textbf{0}} {\left\| {AP'\boldsymbol{\lambda}  - \textbf{x}} \right\|_2}. \quad (2)$$

Note that (2) typically contains a non-PSD matrix Q, thus preventing the solver calculating the correct $\boldsymbol{\lambda}$.

In the end, we return to the flow conservation expression in CDC16 paper; that is
$$\mathcal{F} = \left\{ {\textbf{x}:\exists {\textbf{x}^{\textbf{w}}} \in \mathbb{R}_ +
    ^{\left| \mathcal{A} \right|} ~\text{s.t.}~\textbf{x} =
    \sum\limits_{\textbf{w} \in \mathcal{W}} {{\textbf{x}^{\textbf{w}}}}
    ,~\textbf{N}{\textbf{x}^{\textbf{w}}} = {\textbf{d}^{\textbf{w}}},~\forall
    \textbf{w} \in \mathcal{W}} \right\}.$$

In [67]:
# load node-link incidence matrix
N = zload('node_link_incidence_Sioux.pkz')

In [68]:
N

array([[ 1.,  1., -1., ...,  0.,  0.,  0.],
       [-1.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0., -1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., -1.],
       [ 0.,  0.,  0., ...,  1.,  1.,  1.]])

In [69]:
# load link counts data
with open('demands_Sioux.json', 'r') as json_file:
    demands_Sioux = json.load(json_file)

In [70]:
demands_Sioux['(1,2)']

100.0

In [71]:
# assert(1==2)

n = 24  # number of nodes
m = 76  # number of links

model = Model("OD_matrix_est_Sioux")

# lam = {}
# for i in range(n+1)[1:]:
#     for j in range(n+1)[1:]:
#         if i != j:
#             key = str(i) + '->' + str(j)
#             lam[key] = model.addVar(name='lam_' + key)
            
x = {}
for k in range(m):
    for i in range(n+1)[1:]:
        for j in range(n+1)[1:]:
            if i != j:
                key = str(k) + '->' + str(i) + '->' + str(j)
                x[key] = model.addVar(name='x_' + key)

model.update() 

In [72]:
# Set objective
obj = 0

# for i in range(n+1)[1:]:
#     for j in range(n+1)[1:]:
#         if i != j:
#             key = str(i) + '->' + str(j)
#             obj += lam[key] * lam[key]
            
model.setObjective(obj)

In [73]:
# # Add constraint: lam >= 0
# for i in range(n+1)[1:]:
#     for j in range(n+1)[1:]:
#         if i != j:
#             key = str(i) + '->' + str(j)
#             key_ = '(' + str(i) + ',' + str(j) + ')'
# #             model.addConstr(lam[key] >= 0)
#             model.addConstr(lam[key] == demands_Sioux[key_])
            
for k in range(m):
    s = 0
    for i in range(n+1)[1:]:
        for j in range(n+1)[1:]:
            if i != j:
                key = str(k) + '->' + str(i) + '->' + str(j)
                s += x[key]
                model.addConstr(x[key] >= 0)
    model.addConstr(s - x_0[k] <= 1e2)
    model.addConstr(x_0[k] - s <= 1e2)
            
for l in range(n):
    for i in range(n+1)[1:]:
        for j in range(n+1)[1:]:
            if i != j:
                key_ = str(i) + '->' + str(j)
                key__ = '(' + str(i) + ',' + str(j) + ')'
                s = 0
                for k in range(m):
                    key = str(k) + '->' + str(i) + '->' + str(j)
                    s += N[l, k] * x[key]      
                if (l+1 == i):
                    model.addConstr(s + demands_Sioux[key__] == 0)
                elif (l+1 == j):
                    model.addConstr(s - demands_Sioux[key__]== 0)
                else:
                    model.addConstr(s == 0)
                
#                 if (i == 1 and j == 2):
#                     print(s)

model.update()

In [74]:
# model.setParam('OutputFlag', False)
model.optimize()

Optimize a model with 55352 rows, 41952 columns and 209760 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [0e+00, 0e+00]
  Bounds range    [0e+00, 0e+00]
  RHS range       [1e+02, 2e+04]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 42580 rows and 0 columns
Presolve time: 0.07s
Presolved: 12772 rows, 42028 columns, 122620 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.991e+04
 Factor NZ  : 7.941e+05 (roughly 30 MBytes of memory)
 Factor Ops : 5.594e+07 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   0.00000000e+00 -1.52000000e+03  1.29e+06 0.00e+00  2.34e+02     0s
   1   0.00000000e+00 -7.03736893e+04  1.34e+05 8.33e-17  2.57e+01     0s
   2   0.00000000e+00 -2.83030061e+04  1.38e+04 9.71e-17  3.12e+00     0s
   3   0.00000000e+00 -1.18256050e

In [75]:
lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
# print('Obj: %g' % obj.getValue())

x_0->1->2 0
x_0->1->3 0
x_0->1->4 0
x_0->1->5 0
x_0->1->6 0
x_0->1->7 0
x_0->1->8 0
x_0->1->9 0
x_0->1->10 0
x_0->1->11 0
x_0->1->12 0
x_0->1->13 0
x_0->1->14 0
x_0->1->15 0
x_0->1->16 0
x_0->1->17 0
x_0->1->18 0
x_0->1->19 0
x_0->1->20 0
x_0->1->21 0
x_0->1->22 0
x_0->1->23 0
x_0->1->24 0
x_0->2->1 100
x_0->2->3 100
x_0->2->4 200
x_0->2->5 0
x_0->2->6 0
x_0->2->7 0
x_0->2->8 0
x_0->2->9 200
x_0->2->10 0
x_0->2->11 200
x_0->2->12 100
x_0->2->13 300
x_0->2->14 100
x_0->2->15 100
x_0->2->16 0
x_0->2->17 0
x_0->2->18 0
x_0->2->19 0
x_0->2->20 0
x_0->2->21 0
x_0->2->22 27.3213
x_0->2->23 0
x_0->2->24 0
x_0->3->1 0
x_0->3->2 0
x_0->3->4 0
x_0->3->5 0
x_0->3->6 0
x_0->3->7 0
x_0->3->8 0
x_0->3->9 0
x_0->3->10 0
x_0->3->11 0
x_0->3->12 0
x_0->3->13 0
x_0->3->14 0
x_0->3->15 0
x_0->3->16 0
x_0->3->17 0
x_0->3->18 0
x_0->3->19 0
x_0->3->20 0
x_0->3->21 0
x_0->3->22 0
x_0->3->23 0
x_0->3->24 0
x_0->4->1 0
x_0->4->2 0
x_0->4->3 0
x_0->4->5 0
x_0->4->6 0
x_0->4->7 0
x_0->4->8 0
x_0->4->9 0
x_0->4-

In [76]:
sum(lam_list[0:551])

4594.6576464564205

In [77]:
# write estimation result to file
n = 24  # number of nodes
with open('OD_demand_matrix_Sioux.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1